# Tutorial Básico de PEFT (Parameter-Efficient Fine-Tuning)

## O que é PEFT?

Parameter-Efficient Fine-Tuning (PEFT) é um conjunto de técnicas para adaptar modelos de linguagem grandes (LLMs) pré-treinados a novas tarefas de forma eficiente. Em vez de treinar todos os parâmetros do modelo (fine-tuning completo), o PEFT se concentra em ajustar apenas um pequeno número de parâmetros ou adicionar um pequeno conjunto de novos parâmetros. Isso torna o processo de fine-tuning muito mais rápido e menos custoso em termos de recursos computacionais, ao mesmo tempo em que mantém ou até melhora o desempenho do modelo na tarefa alvo.

## Por que usar PEFT?

Existem várias vantagens em usar técnicas PEFT:

- **Custo computacional reduzido:** Treinar apenas uma pequena fração dos parâmetros requer significativamente menos poder de processamento (GPUs) e tempo.
- **Menor uso de memória:** Como menos parâmetros são atualizados, a quantidade de memória RAM e VRAM necessária é drasticamente reduzida.
- **Menor espaço de armazenamento para checkpoints:** Os checkpoints do modelo ajustado são muito menores, pois apenas os parâmetros modificados ou adicionados precisam ser salvos.
- **Melhor desempenho em tarefas downstream com menos dados:** Em cenários com dados limitados para a tarefa específica, o PEFT pode superar o fine-tuning completo, pois reduz o risco de overfitting ao ajustar menos parâmetros.
- **Compartilhamento e implantação mais fáceis:** Modelos menores e mais eficientes são mais fáceis de compartilhar e implantar em produção.
- **Preservação do conhecimento do modelo pré-treinado:** Ao congelar a maioria dos pesos do modelo original, o PEFT ajuda a reter o vasto conhecimento aprendido durante a fase de pré-treinamento, evitando o "esquecimento catastrófico".

## Visão Geral de Técnicas PEFT Comuns

Existem diversas técnicas PEFT, cada uma com sua abordagem particular para tornar o fine-tuning mais eficiente. Algumas das mais comuns incluem:

- **LoRA (Low-Rank Adaptation):** Congela os pesos pré-treinados do modelo e injeta matrizes de "decomposição de posto baixo" (low-rank) treináveis em cada camada da arquitetura do Transformer. Isso reduz significativamente o número de parâmetros treináveis.
- **Prefix Tuning:** Adiciona um pequeno conjunto de vetores (prefixos) treináveis à entrada de cada camada do Transformer, sem modificar os parâmetros originais do modelo. Esses prefixos são otimizados para guiar o comportamento do modelo na tarefa específica.
- **P-Tuning (Prompt Tuning with Prompt Encoder):** Semelhante ao Prefix Tuning, mas utiliza um pequeno codificador de prompt (geralmente uma MLP) para gerar os prefixos contínuos, oferecendo mais flexibilidade.
- **Prompt Tuning:** É a forma mais simples, onde apenas alguns tokens de prompt "virtuais" ou "contínuos" são adicionados à sequência de entrada e otimizados diretamente, mantendo todo o restante do modelo congelado.

## Instalação

A biblioteca `peft` pode ser facilmente instalada usando o `pip`. Execute o comando abaixo em sua célula de código para instalá-la:

In [ ]:
!pip install peft

## Demonstração de Uso Básico do PEFT com LoRA

### Configuração Inicial e Importações

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, TaskType, AutoPeftModelForSequenceClassification
from datasets import load_dataset

### Carregando o Modelo Base e o Tokenizador

Vamos escolher um modelo pré-treinado da Hugging Face Hub. Para este tutorial, usaremos o `distilbert-base-uncased` por ser um modelo leve e rápido para treinar. É crucial usar o tokenizador correspondente ao modelo para garantir que a entrada seja processada da maneira que o modelo espera.

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Carregamos o modelo para classificação de sequência com num_labels=2, pois o dataset IMDB é para classificação binária (positivo/negativo).
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

`num_labels=2` é definido porque o dataset IMDB, que usaremos a seguir, é para uma tarefa de classificação binária (sentimento positivo ou negativo).

### Preparando o Conjunto de Dados para Classificação de Texto

Usaremos o dataset `imdb` da biblioteca `datasets`, que contém avaliações de filmes e seus respectivos sentimentos. Para agilizar o processo no tutorial, carregaremos apenas uma pequena fração (1%) do conjunto de treino.

In [ ]:
# Carrega uma pequena fração do dataset IMDB (1% do treino)
dataset = load_dataset("imdb", split="train[:1%]")

# Divide o dataset carregado em treino (80%) e teste (20%)
dataset = dataset.train_test_split(test_size=0.2)

# Função para tokenizar os textos
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Aplica a tokenização ao dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove a coluna de texto original, pois não é mais necessária após a tokenização
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

# Renomeia a coluna 'label' para 'labels', que é o nome esperado pelo modelo/Trainer
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# Define o formato do dataset para tensores PyTorch
tokenized_datasets.set_format("torch")

# Cria um DataCollator para preencher dinamicamente as sequências no batch ao comprimento máximo do batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

**Passos da preparação do dataset:**
1. `load_dataset("imdb", split="train[:1%]")`: Carrega os primeiros 1% dos dados de treino do IMDB.
2. `dataset.train_test_split(test_size=0.2)`: Divide essa pequena porção em conjuntos de treino e teste.
3. `tokenize_function`: Define uma função para tokenizar as amostras de texto. `padding="max_length"` e `truncation=True` garantem que todas as sequências tenham o mesmo comprimento.
4. `dataset.map(tokenize_function, batched=True)`: Aplica a tokenização a todas as amostras de forma eficiente.
5. `remove_columns(["text"])`: Remove a coluna de texto original, pois agora temos os `input_ids` e `attention_mask`.
6. `rename_column("label", "labels")`: Renomeia a coluna de rótulos para o nome esperado pelo `Trainer`.
7. `set_format("torch")`: Converte o dataset para o formato de tensores PyTorch.
8. `DataCollatorWithPadding`: Cria um objeto que irá agrupar as amostras em lotes (batches) e aplicar padding dinâmico a cada lote durante o treinamento.

### Definindo a Configuração PEFT (LoRA)

Agora, vamos configurar o LoRA. A `LoraConfig` da biblioteca `peft` nos permite especificar como o LoRA deve ser aplicado. Os parâmetros importantes são:
- `task_type`: O tipo de tarefa para a qual o modelo está sendo adaptado (ex: `SEQ_CLS` para classificação de sequência).
- `r`: A dimensão do rank (posto) das matrizes de atualização LoRA. Um valor menor de `r` significa menos parâmetros treináveis.
- `lora_alpha`: O fator de escala para as matrizes LoRA. É comum definir `lora_alpha` como o dobro de `r`.
- `lora_dropout`: A probabilidade de dropout nas camadas LoRA.
- `base_model_name_or_path`: O nome ou caminho do modelo base, necessário para algumas funcionalidades como salvar o adaptador.

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, 
    r=8, 
    lora_alpha=16, 
    lora_dropout=0.1,
    base_model_name_or_path=model_name # Adicionado para melhor compatibilidade ao salvar
)

# TaskType.SEQ_CLS é usado pois estamos fazendo classificação de sequência (sentimento de texto).

`TaskType.SEQ_CLS` indica que estamos realizando uma tarefa de classificação de sequência (neste caso, análise de sentimento).

### Envolvendo o Modelo Base com PEFT

In [ ]:
peft_model = get_peft_model(model, peft_config)
# Agora `peft_model` é o modelo que será treinado. Apenas os parâmetros LoRA serão atualizados.

### Verificando a Eficiência: Parâmetros Treináveis

In [ ]:
peft_model.print_trainable_parameters()

Observe a saída do comando acima. Ela mostrará o número total de parâmetros no modelo e quantos deles são treináveis. Com LoRA, o número de parâmetros treináveis é uma pequena fração do total, o que demonstra a eficiência da técnica.

### Treinando o Modelo PEFT

Para o treinamento, usamos a classe `Trainer` da biblioteca `transformers`. Precisamos definir `TrainingArguments` que especificam vários hiperparâmetros e configurações para o ciclo de treinamento, como taxa de aprendizado, número de épocas, tamanho do batch, etc.

In [ ]:
training_args = TrainingArguments(
    output_dir="peft_basics_output",          # Diretório para salvar os checkpoints e logs
    learning_rate=2e-3,                     # Taxa de aprendizado (geralmente maior para PEFT)
    per_device_train_batch_size=16,         # Tamanho do batch de treino por dispositivo
    per_device_eval_batch_size=16,          # Tamanho do batch de avaliação por dispositivo
    num_train_epochs=2,                     # Número de épocas de treinamento (pequeno para o tutorial)
    weight_decay=0.01,                      # Força da regularização L2
    evaluation_strategy="epoch",            # Avaliar ao final de cada época
    save_strategy="epoch",                  # Salvar o modelo ao final de cada época
    load_best_model_at_end=True,            # Carregar o melhor modelo ao final do treino
)

trainer = Trainer(
    model=peft_model,                         # O modelo PEFT para treinar
    args=training_args,                       # Argumentos de treinamento
    train_dataset=tokenized_datasets["train"], # Dataset de treino
    eval_dataset=tokenized_datasets["test"],  # Dataset de avaliação/teste
    tokenizer=tokenizer,                      # Tokenizador (para padding e outras funções)
    data_collator=data_collator,              # Data collator para criar os batches
)

# Inicia o treinamento
trainer.train()

# Este processo pode levar alguns minutos, dependendo da sua máquina e da fração do dataset.

O treinamento pode levar alguns minutos para ser concluído. A taxa de aprendizado (`learning_rate`) para PEFT costuma ser mais alta do que para o fine-tuning completo.

### Inferência com o Modelo PEFT Treinado

In [ ]:
text = "This is a fantastic movie!"
inputs = tokenizer(text, return_tensors="pt")

# Garante que os inputs estão no mesmo device que o modelo (CPU ou GPU)
# Se você estiver usando GPU, o peft_model.device será algo como 'cuda:0'
inputs = {k: v.to(peft_model.device) for k, v in inputs.items()}

with torch.no_grad(): # Desativa o cálculo de gradientes para inferência
    logits = peft_model(**inputs).logits

predictions = torch.argmax(logits, dim=-1)

print(f"Input: {text}")
print(f"Predicted label: {'positive' if predictions.item() == 1 else 'negative'}")

**Explicação do código de inferência:**
1. Tokenizamos um texto de exemplo.
2. Movemos os tensores de entrada para o mesmo dispositivo onde o `peft_model` está (CPU ou GPU). Isso é importante para evitar erros de dispositivo.
3. Usamos `torch.no_grad()` para desabilitar o cálculo de gradientes, pois não estamos treinando, apenas fazendo uma predição. Isso economiza memória e acelera o processo.
4. Passamos os inputs para o `peft_model` e obtemos os `logits` (saídas brutas antes da função de ativação final).
5. `torch.argmax` encontra o índice da classe com a maior probabilidade (0 para negativo, 1 para positivo no caso do IMDB).
6. Imprimimos o resultado.

### Salvando e Carregando Adaptadores PEFT

Uma grande vantagem do PEFT é que você só precisa salvar os pesos do adaptador treinado, que são muito pequenos em comparação com o modelo completo. A biblioteca `peft` facilita isso.

In [ ]:
peft_model_path = "./peft_adapter_imdb"

# Salva apenas os pesos do adaptador LoRA treinados
peft_model.save_pretrained(peft_model_path)

print(f"Adaptador PEFT salvo em {peft_model_path}")

# Para carregar o adaptador, você pode usar AutoPeftModelForSequenceClassification
# Isso carregará automaticamente o modelo base correto (especificado na LoraConfig) e aplicará o adaptador.

loaded_model = AutoPeftModelForSequenceClassification.from_pretrained(peft_model_path)

# Vamos testar o modelo carregado
test_text_loaded = "This movie was absolutely terrible and boring."
inputs_loaded = tokenizer(test_text_loaded, return_tensors="pt")

# Garante que os inputs estão no mesmo device que o modelo carregado
inputs_loaded = {k: v.to(loaded_model.device) for k, v in inputs_loaded.items()}

with torch.no_grad():
    logits_loaded = loaded_model(**inputs_loaded).logits

predictions_loaded = torch.argmax(logits_loaded, dim=-1)

print(f"Input: {test_text_loaded}")
print(f"Predicted label (loaded model): {'positive' if predictions_loaded.item() == 1 else 'negative'}")

Ao usar `AutoPeftModelForSequenceClassification.from_pretrained(peft_model_path)`, a biblioteca `peft` carrega automaticamente o modelo base especificado na configuração do adaptador (que foi salva junto com os pesos do adaptador) e, em seguida, aplica os pesos do adaptador LoRA a ele. Isso simplifica o processo de implantação e compartilhamento de modelos ajustados com PEFT.

## Outras Técnicas PEFT

Além do LoRA, a biblioteca `peft` suporta várias outras técnicas eficientes de fine-tuning. A abordagem geral para usá-las é semelhante: você define um objeto de configuração específico da técnica (por exemplo, `PrefixTuningConfig`, `PromptEncoderConfig` ou `PromptTuningConfig`) e depois usa a função `get_peft_model` para aplicar essa configuração ao seu modelo base. Os parâmetros específicos dentro dessas configurações irão variar de acordo com a técnica.

Aqui estão algumas outras técnicas populares:

- **Prefix Tuning:** Esta técnica mantém os parâmetros do modelo original congelados e adiciona um pequeno conjunto de vetores (prefixos) treináveis à entrada de cada camada do Transformer. Esses prefixos são otimizados para guiar o comportamento do modelo na tarefa específica. Diferentemente do LoRA, que modifica as camadas existentes, o Prefix Tuning adiciona novos parâmetros na forma de prefixos.

- **P-Tuning (Prompt Tuning with Prompt Encoder):** Similar ao Prefix Tuning, o P-Tuning também adiciona embeddings treináveis (prompts virtuais ou contínuos) à sequência de entrada. No entanto, em vez de adicionar prefixos fixos a cada camada, o P-Tuning (especialmente a versão com codificador, P-Tuning v2) pode usar um pequeno codificador de prompt (geralmente uma MLP - Multi-Layer Perceptron) para gerar esses embeddings de prompt de forma mais dinâmica e, potencialmente, aplicá-los em diferentes locais do modelo.

- **Prompt Tuning:** Esta é uma forma mais simples e leve de P-Tuning. Ela adiciona um pequeno número de tokens de prompt "virtuais" ou "suaves" (soft prompts) treináveis diretamente aos embeddings de entrada da sequência. Apenas esses embeddings de prompt são otimizados, enquanto todo o resto do modelo pré-treinado permanece congelado. É uma das formas mais eficientes em termos de parâmetros, pois treina ainda menos parâmetros que o LoRA ou Prefix Tuning.

A escolha da técnica PEFT pode depender da tarefa específica, do modelo base e dos recursos computacionais disponíveis. Cada uma oferece um trade-off diferente entre eficiência de parâmetros, desempenho e complexidade de implementação.

## Conclusão e Próximos Passos

Neste tutorial, exploramos os conceitos básicos do Parameter-Efficient Fine-Tuning (PEFT) e demonstramos como usar a técnica LoRA para adaptar um modelo pré-treinado a uma tarefa de classificação de texto com um custo computacional significativamente reduzido. 

**Principais Vantagens do PEFT Recapituladas:**
- **Eficiência de Parâmetros:** Apenas uma pequena fração dos parâmetros do modelo é treinada, resultando em checkpoints muito menores.
- **Redução de Custos Computacionais:** Menos parâmetros para treinar significam menor tempo de treinamento e menor necessidade de GPUs potentes.
- **Menor Uso de Memória:** Tanto para treinamento quanto para inferência.
- **Preservação do Conhecimento:** Evita o esquecimento catastrófico ao manter a maioria dos pesos do modelo original congelados.
- **Versatilidade:** Aplicável a uma ampla gama de modelos e tarefas.

**Próximos Passos:**

1.  **Experimente!** A melhor maneira de aprender é praticando. Tente aplicar LoRA ou outras técnicas PEFT a diferentes modelos e datasets. Ajuste hiperparâmetros como `r`, `lora_alpha`, `learning_rate`, etc., para observar o impacto no desempenho e na eficiência.
2.  **Explore Outras Técnicas PEFT:** Investigue Prefix Tuning, P-Tuning, e Prompt Tuning usando a biblioteca `peft`. Cada uma tem suas nuances e pode ser mais adequada para determinados cenários.
3.  **Aprofunde-se na Documentação:** Consulte os recursos oficiais para entender melhor as capacidades avançadas e as últimas atualizações.

**Recursos Adicionais:**

- **Documentação Oficial do PEFT (Hugging Face):** [`https://huggingface.co/docs/peft`](https://huggingface.co/docs/peft)
- **Repositório GitHub do PEFT:** [`https://github.com/huggingface/peft`](https://github.com/huggingface/peft)
- **Blog da Hugging Face (procure por posts sobre PEFT):** [`https://huggingface.co/blog`](https://huggingface.co/blog)
- **Tarefas de exemplo no repositório PEFT:** O repositório GitHub do PEFT geralmente contém scripts de exemplo para diferentes tarefas e modelos.

PEFT está democratizando o acesso ao fine-tuning de modelos grandes, permitindo que mais pesquisadores e desenvolvedores adaptem esses poderosos modelos às suas necessidades específicas. Continue explorando e construindo!